In [ ]:
!pip install langchain_community langchain_chroma langchain-huggingface langchain torch xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 80.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.0/607.0 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 408.0/408.0 kB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.9/296.9 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 25.7 MB/s eta 

In [ ]:
from huggingface_hub import interpreter_login
from tqdm import tqdm
import torch
import numpy as np
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace, HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, JSONLoader, TextLoader
from langchain_community.document_loaders.merge import MergedDataLoader
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
embed_model_name = "dunzhang/stella_en_400M_v5" #sentence-transformers/sentence-t5-large" #"google/flan-t5-large" #
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_fn = HuggingFaceEmbeddings(
    model_name=embed_model_name,     # Provide the pre-trained model's path
    model_kwargs={'device': device, 'trust_remote_code': True}, # Pass the model configuration options
)

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
llm = HuggingFaceEndpoint(
    repo_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat = ChatHuggingFace(llm=llm, verbose=True)

In [ ]:
vectorstore = Chroma(embedding_function=embedding_fn , collection_name='stella', persist_directory='chroma')

In [ ]:
!rm -rf NLP-System-From-Scratch
!git clone https://github.com/manan-s/NLP-System-From-Scratch

Cloning into 'NLP-System-From-Scratch'...
remote: Enumerating objects: 1917, done.
remote: Counting objects: 100% (1917/1917), done.
remote: Compressing objects: 100% (1277/1277), done.
remote: Total 1917 (delta 888), reused 1571 (delta 632), pack-reused 0 (from 0)
Receiving objects: 100% (1917/1917), 21.28 MiB | 17.68 MiB/s, done.
Resolving deltas: 100% (888/888), done.


The below code snippet creates chroma db's collections. We can select different values of chunk_size and chunk_overlap to create different versions.

In [ ]:
txt_loader = DirectoryLoader('NLP-System-From-Scratch/crawling/scraped_data', glob="**/*.txt", loader_cls=TextLoader)
loader_full = MergedDataLoader(loaders=[txt_loader]) #, pdf_loader])
docs = loader_full.load()
print('Loading folder of {} documents'.format(len(docs)))
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=50)
docs = text_splitter.split_documents(docs)
print('Chunked in {} documents'.format(len(docs)))
vectorstore = Chroma.from_documents(documents=docs, embedding=embedding_fn , collection_name='stella', persist_directory='chroma_stella_joint')

Loading folder of 1569 documents
Chunked in 16923 documents
